## Bobby Darin Lyrics with an RNN

Adapted from the [tensorflow example](https://www.tensorflow.org/tutorials/sequences/text_generation), to run on [datahub.ucsd.edu](datahub.ucsd.edu), with TF 1.14. 

Joseph Chang, jdchang@ucsd.edu.

## Setup

### Import TensorFlow and other libraries

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
from IPython.display import Image


### Access Bobby Darin dataset

In [2]:
# Example training on Kanye West
path_to_file = "bobby-darin-lyrics.txt"


### Read the data

First, look in the text.

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding = "ISO-8859-1")
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 247500 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

one, two, three!

the stars
are gonna twinkle and shine
this evenin'
'bout a quarter to nine

my lovin' arms
are gonna tenderly twine
this evenin' around you
quarter to nine

i know i won't be late
'cause at half past eight
i'll be waitin' there
i'll


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

29 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [7]:
print('{')
for char,_ in zip(char2idx, range(len(char2idx))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  'a' :   6,
  'b' :   7,
  'c' :   8,
  'd' :   9,
  'e' :  10,
  'f' :  11,
  'g' :  12,
  'h' :  13,
  'i' :  14,
  'k' :  15,
  'l' :  16,
  'm' :  17,
  'n' :  18,
  'o' :  19,
  'p' :  20,
  'q' :  21,
  'r' :  22,
  's' :  23,
  't' :  24,
  'u' :  25,
  'v' :  26,
  'w' :  27,
  'y' :  28,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'one, two, thr' ---- characters mapped to int ---- > [19 18 10  4  1 24 27 19  4  1 24 13 22]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text. 

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right. 

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

o
n
e
,
 


The `batch` method lets us easily convert these individual characters to sequences of the desired size. 

In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"one, two, three!\n\nthe stars\nare gonna twinkle and shine\nthis evenin'\n'bout a quarter to nine\n\nmy lovi"
"n' arms\nare gonna tenderly twine\nthis evenin' around you\nquarter to nine\n\ni know i won't be late\n'cau"
"se at half past eight\ni'll be waitin' there\ni'll be waitin' where the lane begins\nwaitin' for you on "
"needles and pins\n\nand then\nthe world is gonna be mine\nthis evenin'\n'bout a quarter to nine\n\nget it!\na"
"hhh, ah\noh!\n\noh, the stars\nare gonna twinkle and shine\nthis very evenin'\n'bout a quarter to nine\n\nbot"


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "one, two, three!\n\nthe stars\nare gonna twinkle and shine\nthis evenin'\n'bout a quarter to nine\n\nmy lov"
Target data: "ne, two, three!\n\nthe stars\nare gonna twinkle and shine\nthis evenin'\n'bout a quarter to nine\n\nmy lovi"


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 19 ('o')
  expected output: 18 ('n')
Step    1
  input: 18 ('n')
  expected output: 10 ('e')
Step    2
  input: 10 ('e')
  expected output: 4 (',')
Step    3
  input: 4 (',')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 24 ('t')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [14]:
# Batch size (number of 100 word sequences to train on)
BATCH_SIZE = 64

# How many individual training steps per samples (steps)
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

### Neuronal Modal


__NOTE__: Insert an image here diagramming basic neural net

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Next define a function to build the model.

Use `CuDNNGRU` if running on GPU.  

In [16]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNLSTM
  rnn2 = tf.keras.layers.CuDNNLSTM
  rnn3 = tf.keras.layers.CuDNNLSTM
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.LSTM, recurrent_activation='sigmoid')

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),

    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next character:

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [19]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 29) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length: 

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           7424      
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (64, None, 1024)          5251072   
_________________________________________________________________
dense (Dense)                (64, None, 29)            29725     
Total params: 5,288,221
Trainable params: 5,288,221
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary. 

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [21]:
# sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1) # TF 1.12
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [22]:
sampled_indices

array([18, 12, 23, 19, 13,  3, 16,  2, 10, 23, 18,  0, 22,  9, 22, 12,  8,
       28, 27,  9, 24, 14, 14, 13,  2, 18,  4, 12, 21,  2,  2, 16, 13, 16,
       28, 25,  6,  8, 28, 28, 17, 26, 28, 21, 17, 15, 14, 27, 25, 21,  3,
       10, 25,  8, 28,  4, 12,  4, 28, 10, 23, 14, 18, 26, 10, 11, 25, 13,
       28, 18, 23, 12,  8, 24,  4, 15, 23, 10,  0,  3, 14, 25, 24, 15,  0,
        7, 28,  7,  8,  4, 10,  3, 19,  1,  5, 12,  5, 27, 25,  3])

Decode these to see the text predicted by this untrained model:

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "twinkle and shine\nthis very evenin'\n'bout a quarter to nine\n\nboth these lovin' arms\nare gonna tender"

Next Char Predictions: 
 "ngsoh'l!esn\nrdrgcywdtiih!n,gq!!lhlyuacyymvyqmkiwuq'eucy,g,yesinvefuhynsgct,kse\n'iutk\nbybc,e'o -g-wu'"


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions. 

Because our model returns logits, we need to set the `from_logits` flag. 


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 29)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.3668737


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.train.AdamOptimizer` with default arguments and the loss function.

NOTE TO SELF: Insert reference about optimizers, gradient descent, and Adam particularly.

In [25]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 3 epochs to train the model. Increase the number of epochs to increase training time.

In [27]:
EPOCHS=30

In [28]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/30
38/38 [==============================] - 3s 81ms/step - loss: 2.8568
Epoch 2/30
38/38 [==============================] - 2s 54ms/step - loss: 1.7944
Epoch 3/30
38/38 [==============================] - 2s 55ms/step - loss: 0.4686
Epoch 4/30
38/38 [==============================] - 2s 56ms/step - loss: 0.2235
Epoch 5/30
38/38 [==============================] - 2s 55ms/step - loss: 0.1739
Epoch 6/30
38/38 [==============================] - 2s 55ms/step - loss: 0.1518
Epoch 7/30
38/38 [==============================] - 2s 54ms/step - loss: 0.1317
Epoch 8/30
38/38 [==============================] - 2s 55ms/step - loss: 0.1200
Epoch 9/30
38/38 [==============================] - 2s 55ms/step - loss: 0.1119
Epoch 10/30
38/38 [==============================] - 2s 54ms/step - loss: 0.1056
Epoch 11/30
38/38 [==============================] - 2s 55ms/step - loss: 0.1002
Epoch 12/30
38/38 [==============================] - 2s 55ms/step - loss: 0.0965
Epoch 13/30
38/38 [==================

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built. 

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training/ckpt_30'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            7424      
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (1, None, 1024)           5251072   
_________________________________________________________________
dense_1 (Dense)              (1, None, 29)             29725     
Total params: 5,288,221
Trainable params: 5,288,221
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a multinomial distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [32]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.05

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
        
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [38]:
text = generate_text(model, start_string=u"hate ")
print(text)

hate world is gonna be mine
this evenin'
'bout a quarter to nine!

thank yo won ly twine
all around you
'round a quarter to nine

i know i won't be late
'cause at half-past eight
i will hurry there
i'm gonna be waitin' where the lane begins
waitin' for you on needles and pis hene
this evenin'
'bout a quarter to nine

my lovin' arms
are gonna tenderly twine
ald is gonna be mine
this evenin'
'bout a quarter to nine

get it!
ahhhh, ah
oh!

oh, the stars
are gonna twinkle and shine
this very evenin'
'bout a quarter to nine

both these lovin' arms
are gonna tenderly twine
all around you
'round a quarter to nine

i know i won't be late
'cause at half past eight
i'll be waitin' there
i'll be waitin' where the lane begins
waitin' on you on needles and pins

and then
the world is gonna be mine
this evenin'
'bout a quarter to nine

get it!
ahhh, ah
oh!

oh, the stars
are gonna twinkle and shine
this evenin'
'bout a quarter to nine

my lovin' arms
are gonna tenderly twine
this evenin' around you


In [34]:
filename = "bobby-darin-lyrics-generated.txt"
if os.path.exists(filename):
  os.remove(filename)
else:
  print('File does not exist to be removed')

In [35]:
# Write lyrics to .txt file
with open(filename, 'a') as lyrics_file:
    lyrics_file.write(text)
    lyrics_file.write('\n\n')
print('Lyric Scraping Completed')

Lyric Scraping Completed


# Activities

1. Gradually increase the number of training epochs, and see how your generated output evolves. "Sample" the model a number of times and store your results.
2. Train on your own corpus, instead of Shakespeare.
  - Find a plain text data source and upload it to datahub.
  - How long does your text need to be? (shakespeare is 1.1M characters)
3. Experiment with the network parameters. In ML we call these "hyperparameters", parameters of the network, to distinguish from the internal weights and biases ("parameters") that are learned during training.
  - How does changing ```temperature = 1.0``` change the qualities of the generated text? E.g. try values 0 < 1.0, > 1.0, near 0.0
  - How does changing the number of embedding dimensions ```embedding_dim = 256``` change the results? This is more important in word-level models.
  - How does changing the number of RNN units (internal "nodes" in the LSTM) change the results? By default we have ```rnn_units = 1024```
  - Does changing the length of training sequences ```seq_length = 100``` change the results? 
  - Add a new layer to your network (copy the rnn statement in the model construction)
4. Future directions: 
  - Move on to a word-level RNN model (like this https://github.com/hunkim/word-rnn-tensorflow)
  - Move on to fine-tuning the GPT-2 model (https://github.com/nshepperd/gpt-2, https://pmbaumgartner.github.io/blog/gpt2-jokes/)